In [1]:
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk
import sqlite3
import os

class BookStoreApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sistem Pembelian Buku")
        
        screen_width = root.winfo_screenwidth()
        screen_height = root.winfo_screenheight()
        self.root.geometry(f"{screen_width}x{screen_height}")

        self.conn = sqlite3.connect('user_data.db')
        self.cursor = self.conn.cursor()
        self.create_table()
        
        self.cart = []  
        self.login_page()

    def create_table(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS users (
                username TEXT PRIMARY KEY,
                password TEXT NOT NULL
            )
        ''')
        self.conn.commit()

    def login_page(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root, bg='White')
        self.frame.pack(fill='both', expand=True)

        img_path = "Junior Books.png"  
        if os.path.exists(img_path):
            login_image = Image.open(img_path)
            login_image = login_image.resize((700, 700))  
            login_photo = ImageTk.PhotoImage(login_image)
            login_label = tk.Label(self.frame, image=login_photo, bg='White')
            login_label.image = login_photo
            login_label.place(x=150, y=50)  

        form_frame = tk.Frame(self.frame, bg='White')
        form_frame.place(x=850, y=270) 

        tk.Label(form_frame, text="Login", bg='White', font=('Helvetica', 34, 'bold')).pack(pady=10)
        tk.Label(form_frame, text="Username", bg='White').pack(pady=5)
        self.username_entry = tk.Entry(form_frame, width=25)
        self.username_entry.pack(pady=5)

        self.username_entry.bind("<Return>", self.focus_password_entry)

        tk.Label(form_frame, text="Password", bg='White').pack(pady=5)
        self.password_entry = tk.Entry(form_frame, show='*', width=25)
        self.password_entry.pack(pady=5)

        self.password_entry.bind("<Return>", self.authenticate)

        tk.Button(form_frame, text="Login", fg='black', bg='white', width=25, height=1, command=self.authenticate).pack(pady=5)
        tk.Button(form_frame, text="Register", fg='black', bg='white', width=25, height=1, command=self.register_page).pack(pady=5)

    def focus_password_entry(self, event):
        self.password_entry.focus()

    def register_page(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root, bg='White')
        self.frame.pack(fill='both', expand=True)

        img_path = "Junior Books.png" 
        if os.path.exists(img_path):
            register_image = Image.open(img_path)
            register_image = register_image.resize((700, 700))  
            register_photo = ImageTk.PhotoImage(register_image)
            register_label = tk.Label(self.frame, image=register_photo, bg='White')
            register_label.image = register_photo
            register_label.place(x=150, y=50)

        form_frame = tk.Frame(self.frame, bg='White')
        form_frame.place(x=850, y=270)

        tk.Label(form_frame, text="Registrasi", bg='White', font=('Helvetica', 34, 'bold')).pack(pady=10)
        tk.Label(form_frame, text="Username", bg='White').pack(pady=5)
        self.reg_username_entry = tk.Entry(form_frame, width=25)
        self.reg_username_entry.pack(pady=5)

        self.reg_username_entry.bind("<Return>", self.focus_reg_password_entry)

        tk.Label(form_frame, text="Password", bg='White').pack(pady=5)
        self.reg_password_entry = tk.Entry(form_frame, show='*', width=25)
        self.reg_password_entry.pack(pady=5)

        self.reg_password_entry.bind("<Return>", self.save_registration)

        tk.Button(form_frame, text="Registrasi", fg='black', bg='white', width=25, height=1, command=self.save_registration).pack(pady=5)
        tk.Button(form_frame, text="Kembali", fg='black', bg='white', width=25, height=1, command=self.login_page).pack(pady=5)

    def focus_reg_password_entry(self, event):
        self.reg_password_entry.focus()

    def save_registration(self, event=None):
        username = self.reg_username_entry.get()
        password = self.reg_password_entry.get()
        
        if username and password:
            try:
                self.cursor.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
                self.conn.commit()
                messagebox.showinfo("Registrasi", "Registrasi berhasil!")
                self.login_page()
            except sqlite3.IntegrityError:
                messagebox.showerror("Registrasi", "Username sudah terdaftar!")
        else:
            messagebox.showerror("Registrasi", "Username dan Password tidak boleh kosong!")

    def authenticate(self, event=None):
        username = self.username_entry.get()
        password = self.password_entry.get()
        
        self.cursor.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password))
        user = self.cursor.fetchone()

        if user:
            self.main_menu()  
        else:
            messagebox.showerror("Login", "Username atau Password salah atau belum terdaftar!")

    def main_menu(self):
        self.clear_frame()
        self.frame = tk.Frame(self.root, bg='White')
        self.frame.pack(fill='both', expand=True)

        tk.Label(self.frame, text="Menu Utama").pack(pady=10)
        tk.Button(self.frame, text="Pilih Produk", command=self.select_product).pack(pady=10)
        tk.Button(self.frame, text="Lihat Keranjang", command=self.view_cart).pack(pady=10)
        tk.Button(self.frame, text="Keluar", command=self.root.quit).pack(pady=10)